## Import Libraries and data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os


In [8]:
#folder path
path = r'C:\Users\kaydo\OneDrive\Desktop\Python\Data\OG Data'

In [4]:
# import orders_products_merged.pkl
ords_prods_all = pd.read_pickle(r'C:\Users\kaydo\OneDrive\Desktop\Python\Data\Prepared Data\orders_products_merged_aggregated.pkl')

In [10]:
# import new customer data set
df_customers = pd.read_csv(os.path.join(path, 'custinfo.csv'),index_col = False)

In [11]:
df_customers.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [12]:
df_customers.shape

(206209, 10)

In [13]:
df_customers.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

## Step Four (Wrangle the Data)

## Drop Columns that are not needed

In [20]:
# Drop Surname & first name since it is necessary
df_customers = df_customers.drop(columns = ['First Name'])

In [21]:
df_customers.columns

Index(['user_id', 'Gender', 'STATE', 'Age', 'date_joined', 'n_dependants',
       'fam_status', 'income'],
      dtype='object')

## Step Five

## Explore Missing values

In [22]:
#check for missing values
df_customers.isnull().sum()

user_id         0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

No missing values are in this dataset.

## Rename columns for better clarity

In [23]:
#rename columns
df_customers.rename(columns = {'Gender' : 'gender', 'STATE': 'state', 'Age' : 'age', 'n_dependants' : 'number_dependants', 'fam_status' : 'family_status'},inplace = True)

In [24]:
#check output 
df_customers.head()

user_id  gender       state  age date_joined  number_dependants  \
0    26711  Female    Missouri   48    1/1/2017                  3   
1    33890  Female  New Mexico   36    1/1/2017                  0   
2    65803    Male       Idaho   35    1/1/2017                  2   
3   125935  Female        Iowa   40    1/1/2017                  0   
4   130797  Female    Maryland   26    1/1/2017                  1   

  family_status  income  
0       married  165665  
1        single   59285  
2       married   99568  
3        single   42049  
4       married   40374

## Checking Data Types


In [25]:
#Check user_id
df_customers['user_id'].dtype

dtype('int64')

In [33]:
#change varibale data type
df_customers['user_id'] = df_customers['user_id'].astype('str')
df_customers['user_id'].dtype

dtype('O')

In [34]:
ords_prods_all['user_id'] = ords_prods_all['user_id'].astype('str')
ords_prods_all['user_id'].dtype

dtype('O')

## Data consistency Checks

In [29]:
df_customers.describe()

age  number_dependants         income
count  206209.000000      206209.000000  206209.000000
mean       49.501646           1.499823   94632.852548
std        18.480962           1.118433   42473.786988
min        18.000000           0.000000   25903.000000
25%        33.000000           0.000000   59874.000000
50%        49.000000           1.000000   93547.000000
75%        66.000000           3.000000  124244.000000
max        81.000000           3.000000  593901.000000

Nothing out of the ordinary appears. Perhaps the max for income but it could absolutely be a true value.

## Find duplicates

In [30]:
#find duplicates
df_customers[df_customers.duplicated()]

Empty DataFrame
Columns: [user_id, gender, state, age, date_joined, number_dependants, family_status, income]
Index: []

No duplicates have been discovered.

## Check for Mixed Type Data

In [31]:
# check for mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

No mixed data types have been found.

## Step Six

## Combine customer data with prepared Instacart data

In [35]:
#merge datasets
df_instacart = pd.merge(ords_prods_all, df_customers, on = 'user_id', how = 'outer')

In [36]:
#check output
df_instacart.head()

order_id user_id eval_set  order_number  Order_day_of_week  \
0   2539329       1    prior             1                  2   
1   2398795       1    prior             2                  3   
2    473747       1    prior             3                  3   
3   2254736       1    prior             4                  4   
4    431534       1    prior             5                  4   

   order_hour_of_day  days_since_prior_order  product_id  add_to_cart_order  \
0                  8                     NaN         196                  1   
1                  7                    15.0         196                  1   
2                 12                    21.0         196                  1   
3                  7                    29.0         196                  1   
4                 15                    28.0         196                  1   

   reordered  ... spending_flag  median_freq        order_freq_flag  gender  \
0          0  ...   Low Spender         20.5  Non-Frequent Customer  Female   
1          1  ...   Low Spender         20.5  Non-Frequent Customer  Female   
2          1  ...   Low Spender         20.5  Non-Frequent Customer  Female   
3          1  ...   Low Spender         20.5  Non-Frequent Customer  Female   
4          1  ...   Low Spender         20.5  Non-Frequent Customer  Female   

     state age date_joined number_dependants family_status  income  
0  Alabama  31   2/17/2019                 3       married   40423  
1  Alabama  31   2/17/2019                 3       married   40423  
2  Alabama  31   2/17/2019                 3       married   40423  
3  Alabama  31   2/17/2019                 3       married   40423  
4  Alabama  31   2/17/2019                 3       married   40423  

[5 rows x 32 columns]

In [38]:
#check value counts
df_instacart['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [39]:
df_instacart.shape

(32404859, 32)

In [40]:
#export to pkl file
df_instacart.to_pickle(r'C:\Users\kaydo\OneDrive\Desktop\Python\Data\Prepared Data\Instacart_data.pkl')